In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml

from sklearn.model_selection import StratifiedShuffleSplit

data= fetch_openml('mnist_784', version=1)#Get data from https://www.openml.org/d/554

In [2]:
dfData = pd.DataFrame(np.c_[data["data"],data["target"]],columns = data["feature_names"]+["target"])
targetNumber = 1
dfData["target"] = dfData["target"].apply(lambda x: 1 if int(x) == targetNumber else 0)

In [3]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in stratSplit.split(dfData[data["feature_names"]], dfData["target"]):
    X_train = dfData[data["feature_names"]].iloc[train_index]
    X_test = dfData[data["feature_names"]].iloc[test_index]
    
    y_train = dfData["target"].iloc[train_index]
    y_test = dfData["target"].iloc[test_index]

In [4]:
from sklearn.linear_model import LogisticRegression
logReg_clf = LogisticRegression()
logReg_clf.fit(X_train, y_train)

y_test_dec_func = logReg_clf.decision_function(X_test)

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [5]:
from sklearn.metrics import precision_recall_fscore_support

def applyThreshold(decFunc, threshold):
    return [0 if x < threshold else 1 for x in decFunc]
threshold = 1
y_test_pred = applyThreshold(y_test_dec_func, threshold)
precision,recall,fscore,support = precision_recall_fscore_support(y_true=y_test, y_pred=y_test_pred)
print("precision:",np.round(precision,2)[1])
print("recall:",np.round(recall,2)[1])

precision: 0.98
recall: 0.95


In [6]:
# Using thresholds from precision recall curve
from sklearn.metrics import precision_recall_curve
yRawScores = logReg_clf.decision_function(X_train)
precisions, recalls, thresholds = precision_recall_curve(y_train, yRawScores)

In [7]:
thresholds # decision function thresholds

array([-16.35698108, -16.35644843, -16.35635661, ...,  11.09427387,
        11.30778078,  11.55189636])

In [8]:
print("precision:",precisions[np.argmax(thresholds>=threshold)])
print("recall:",recalls[np.argmax(thresholds>=threshold)])

precision: 0.9877690802348337
recall: 0.9611234528721041


In [9]:
dfData = pd.DataFrame(np.c_[data["data"],data["target"]],columns = data["feature_names"]+["target"])
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in stratSplit.split(dfData[data["feature_names"]], dfData["target"]):
    X_train_multi = dfData[data["feature_names"]].iloc[train_index]
    X_test_multi = dfData[data["feature_names"]].iloc[test_index]
    
    y_train_multi = dfData["target"].iloc[train_index]
    y_test_multi = dfData["target"].iloc[test_index]
logReg_multi_clf = LogisticRegression()
logReg_multi_clf.fit(X_train_multi, y_train_multi)

y_test_multi_dec_func = logReg_multi_clf.decision_function(X_test_multi)

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [10]:
y_test_multi_dec_func.shape, logReg_multi_clf.classes_

((14000, 10),
 array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object))

In [11]:
y_test_multi_dec_func = logReg_multi_clf.decision_function(X_test_multi)
targetClass = '1'
targClassInd = np.argmax(logReg_multi_clf.classes_==targetClass)
precisions, recalls, thresholds = precision_recall_curve(y_test_multi,
                                                         y_test_multi_dec_func[:, targClassInd],
                                                         pos_label=targetClass # label of target class in y_true
                                                        )
print("precision:",precisions[np.argmax(thresholds>=threshold)])
print("recall:",recalls[np.argmax(thresholds>=threshold)])

precision: 0.6032295271049596
recall: 0.9961904761904762


In [13]:
labelMap = {}
for label in logReg_multi_clf.classes_:
    labelMap[label] = np.argmax(logReg_multi_clf.classes_==label)
y_test_multi_dec_func = logReg_multi_clf.decision_function(X_test_multi)
y_test_multi_pred = applyThreshold(y_test_multi_dec_func[:, targClassInd], threshold)
precision,recall,fscore,support = precision_recall_fscore_support(y_true=[labelMap[x] for x in y_test_multi],
                                                                  y_pred=y_test_multi_pred,
                                                                  pos_label=labelMap[targetClass]
                                                                 )
print("precision:",np.round(precision,2)[1])
print("recall:",np.round(recall,2)[1])

precision: 0.6
recall: 1.0


/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
